In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem

In [2]:
from scipy import stats

In [3]:
import pandas as pd, numpy as np, re, os
from src.evaluation import Evaluator, ResultsFromMeanStdDF
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 12})

In [4]:
## This cell would need for testing phase - Ignore this
%load_ext autoreload
%autoreload 2

In [5]:
paths = [ os.path.join(dataset.name, experiment.name) 
         for dataset in os.scandir('experiments') if dataset.is_dir() and 'chembl' in dataset.name 
         for experiment in os.scandir(dataset.path)  ]

In [6]:
paths=[]
for dataset in os.scandir('experiments'):
    if not dataset.is_dir() or not 'chembl' in dataset.name or 'atom-fragSMILES' in dataset.name:
        continue

    if 'chemblAug5' in dataset.name:
        for setup in os.scandir(dataset):
            if '2hl_512hu' in setup.name and '256bs' in setup.name:
                paths.append(setup.path)

    elif 'chembl' in dataset.name:
        for setup in os.scandir(dataset):
            if '2hl_512hu' in setup.name and '512bs' in setup.name:
                paths.append(setup.path)

In [7]:
paths

['experiments/chemblAug5_SELFIES-RNN/2hl_512hu_256bs_0.001lr',
 'experiments/chemblAug5_SMILES-RNN/2hl_512hu_256bs_0.001lr',
 'experiments/chemblAug5_fragSMILES-RNN/2hl_512hu_256bs_300es_0.001lr',
 'experiments/chembl_SELFIES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chembl_SMILES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chembl_fragSMILES-RNN/2hl_512hu_512bs_300es_0.001lr',
 'experiments/chemblAug5_t-SMILES-RNN/2hl_512hu_256bs_0.001lr',
 'experiments/chembl_t-SMILES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chembl_atom-fragSMILES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chemblAug5_atom-fragSMILES-RNN/2hl_512hu_256bs_0.001lr']

In [17]:
evaluators = [Evaluator(path) for path in paths]

In [18]:
evaluators_table = pd.concat([evaluator.get_as_DFcell() for evaluator in evaluators ], ignore_index=True, axis=0)

In [19]:
evaluators_table = evaluators_table.drop(columns=['dataset','notation']).groupby(['aug','hl','hu','bs','lr']).agg(lambda x: x.dropna())

In [20]:
evaluators_table

,,,,,SELFIES,SMILES,fragSMILES,t-SMILES,atom-fragSMILES
aug,hl,hu,bs,lr,,,,,
1,2,512,512,0.001,<src.evaluation.Evaluator object at 0x7ffa50d6...,<src.evaluation.Evaluator object at 0x7ffa50d6...,<src.evaluation.Evaluator object at 0x7ffa50ec...,<src.evaluation.Evaluator object at 0x7ffa50ec...,<src.evaluation.Evaluator object at 0x7ffa50ec...
5,2,512,256,0.001,<src.evaluation.Evaluator object at 0x7ffa50d6...,<src.evaluation.Evaluator object at 0x7ffa510c...,<src.evaluation.Evaluator object at 0x7ffa5192...,<src.evaluation.Evaluator object at 0x7ffa50ec...,<src.evaluation.Evaluator object at 0x7ffa50d6...


In [21]:
evaluators_table.map(lambda x: x.load_novels())

,,,,,SELFIES,SMILES,fragSMILES,t-SMILES,atom-fragSMILES
aug,hl,hu,bs,lr,,,,,
1,2,512,512,0.001,True,True,True,True,True
5,2,512,256,0.001,True,True,True,True,True


In [22]:
evaluators_table.map(lambda x: x.load_scaffolds())

,,,,,SELFIES,SMILES,fragSMILES,t-SMILES,atom-fragSMILES
aug,hl,hu,bs,lr,,,,,
1,2,512,512,0.001,True,True,True,True,True
5,2,512,256,0.001,True,True,True,True,True


## scaffolds analysis

In [23]:
scaffDFs=pd.concat( [evaluator.getScaffoldResults() for evaluator in evaluators if not evaluator.novels is None], ignore_index=True)

[13:42:31] WARNING: not removing hydrogen atom without neighbors
[13:42:31] WARNING: not removing hydrogen atom without neighbors
[13:42:40] non-ring atom 1 marked aromatic


In [24]:
scaffDFs['unique%']=scaffDFs['unique']/scaffDFs['total']
scaffDFs['novel%']=scaffDFs['novel']/scaffDFs['unique']
scaffDFs['chiral%']=scaffDFs['chiral']/scaffDFs['novel']

In [25]:
scaffDFs

,fold,amount,epoch,T,total,unique,novel,chiral,dataset,aug,notation,hl,hu,bs,lr,unique%,novel%,chiral%
0,0,6000,3,1.0,5972,5675,5316,917,chembl,5,SELFIES,2,512,256,0.001,0.950268,0.936740,0.172498
1,1,6000,3,1.0,5974,5598,5214,874,chembl,5,SELFIES,2,512,256,0.001,0.937061,0.931404,0.167626
2,2,6000,3,1.0,5987,5614,5206,924,chembl,5,SELFIES,2,512,256,0.001,0.937698,0.927325,0.177488
3,3,6000,3,1.0,5981,5651,5279,861,chembl,5,SELFIES,2,512,256,0.001,0.944825,0.934171,0.163099
4,4,6000,3,1.0,5976,5585,5178,775,chembl,5,SELFIES,2,512,256,0.001,0.934572,0.927126,0.149672
5,0,6000,3,1.0,5975,5513,4840,845,chembl,5,SMILES,2,512,256,0.001,0.922678,0.877925,0.174587
6,1,6000,3,1.0,5968,5600,5020,874,chembl,5,SMILES,2,512,256,0.001,0.938338,0.896429,0.174104
7,2,6000,3,1.0,5979,5634,5090,866,chembl,5,SMILES,2,512,256,0.001,0.942298,0.903443,0.170138
8,3,6000,3,1.0,5975,5698,5174,1145,chembl,5,SMILES,2,512,256,0.001,0.953640,0.908038,0.221299
9,4,6000,3,1.0,5981,5588,5011,1041,chembl,5,SMILES,2,512,256,0.001,0.934292,0.896743,0.207743


## Count how many scaffolds are novel cause of new cyclic fragments

In [26]:
from src.processer import fragmentSmiles, applyFncPool

In [27]:
vocabs=[]
train_data = pd.read_csv(f'data/chembl.tar.xz', usecols = ['fragsmiles'] + [f'fold{fold}' for fold in range(5)], compression="xz")
train_data['fragsmiles'] = train_data['fragsmiles'].str.split(' ')
for fold in range(5):
    query = f'fold{fold} == "train"'
    fold_data = train_data.query(query)['fragsmiles']
    vocabs.append(np.unique(np.concatenate(fold_data.values)))

In [28]:
def checkScaffoldFragments(scaffolds, vocab):

    def checkNewFrag(frags):
        mask = np.in1d(frags, vocab, invert=True)

        ## If u want track which new fragments are
        # new = np.array(frags)[mask]
        # if len(new)>0:
        #     print(new)
        return mask.any()
    
    scaffFrags = pd.Series( applyFncPool(scaffolds, fnc=fragmentSmiles) ).str.split(' ')
    maskNewFrags=scaffFrags.apply(checkNewFrag)

    # return maskNewFrags
    return maskNewFrags.sum()


In [29]:
def countNewScaffCauseFrags(evaluator):
    results = evaluator.novels.copy()

    results['inducedNewFrags']=results.apply(lambda x: checkScaffoldFragments(x.sampled.novelScaff, vocabs[x.fold]), axis=1)
    # return results.apply(lambda x: checkScaffoldFragments(x.sampled.novelScaff, vocabs[x.fold]), axis=1)

    setup = {k:v for k,v in evaluator.setupArgs.items() if k!='es'}
    results = results.assign(**evaluator.datasetArgs, **setup)

    results.drop(columns='sampled', inplace=True)
    return results.sort_values(['amount','fold','epoch'])

In [30]:
fragInduced = pd.concat( [ countNewScaffCauseFrags(evaluator) for evaluator in evaluators if evaluator.datasetArgs['notation']!='fragSMILES'], ignore_index=True)

In [31]:
induced = scaffDFs.merge(fragInduced, how='outer')

In [32]:
induced

,fold,amount,epoch,T,total,unique,novel,chiral,dataset,aug,notation,hl,hu,bs,lr,unique%,novel%,chiral%,inducedNewFrags
0,0,6000,3,1.0,5972,5675,5316,917,chembl,5,SELFIES,2,512,256,0.001,0.950268,0.936740,0.172498,2371.0
1,0,6000,3,1.0,5975,5513,4840,845,chembl,5,SMILES,2,512,256,0.001,0.922678,0.877925,0.174587,1029.0
2,0,6000,3,1.0,5934,5363,4685,824,chembl,5,t-SMILES,2,512,256,0.001,0.903775,0.873578,0.175880,526.0
3,0,6000,4,1.0,5975,5412,4281,845,chembl,5,fragSMILES,2,512,256,0.001,0.905774,0.791020,0.197384,NaN
4,0,6000,10,1.0,5968,5584,5170,761,chembl,1,SELFIES,2,512,512,0.001,0.935657,0.925860,0.147195,2768.0
5,0,6000,10,1.0,5983,5602,4760,822,chembl,1,SMILES,2,512,512,0.001,0.936320,0.849697,0.172689,849.0
6,0,6000,10,1.0,5912,5340,4735,802,chembl,1,t-SMILES,2,512,512,0.001,0.903248,0.886704,0.169377,598.0
7,0,6000,12,1.0,5982,5524,4652,546,chembl,1,atom-fragSMILES,2,512,512,0.001,0.923437,0.842143,0.117369,163.0
8,0,6000,12,1.0,5920,5383,4809,824,chembl,1,t-SMILES,2,512,512,0.001,0.909291,0.893368,0.171345,689.0
9,0,6000,16,1.0,5989,5527,4526,776,chembl,1,fragSMILES,2,512,512,0.001,0.922859,0.818889,0.171454,NaN


In [33]:
induced['inducedNewFrags%']=induced['inducedNewFrags']/induced['novel']

In [34]:
inducedGrp=induced.drop(columns=['dataset','T','fold','amount','epoch'])\
                    .groupby(['aug','notation','hl','hu','bs','lr'], as_index=True, dropna=False)

In [35]:
inducedStats=inducedGrp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ]

In [36]:
inducedStats.fillna(0, inplace=True)

In [37]:
inducedStats

total                 unique  \
                                             mean       std         mean   
aug notation        hl hu  bs  lr                                          
1   SELFIES         2  512 512 0.001  5964.000000  5.522681  5572.600000   
    SMILES          2  512 512 0.001  5976.200000  7.726578  5489.200000   
    atom-fragSMILES 2  512 512 0.001  5978.800000  6.534524  5521.000000   
    fragSMILES      2  512 512 0.001  5985.000000  8.366600  5495.600000   
    t-SMILES        2  512 512 0.001  5908.166667  8.681398  5331.833333   
5   SELFIES         2  512 256 0.001  5978.000000  6.041523  5624.600000   
    SMILES          2  512 256 0.001  5975.600000  4.979960  5606.600000   
    atom-fragSMILES 2  512 256 0.001  5982.333333  6.110101  5487.666667   
    fragSMILES      2  512 256 0.001  5978.000000  6.855655  5511.600000   
    t-SMILES        2  512 256 0.001  5924.600000  9.633276  5379.600000   

                                                       novel              \
                                            std         mean         std   
aug notation        hl hu  bs  lr                                          
1   SELFIES         2  512 512 0.001  42.881231  5155.000000   72.273785   
    SMILES          2  512 512 0.001  79.162491  4599.600000  112.717789   
    atom-fragSMILES 2  512 512 0.001  44.620623  4648.600000   84.019641   
    fragSMILES      2  512 512 0.001  52.074946  4443.800000   67.236151   
    t-SMILES        2  512 512 0.001  45.128336  4732.166667   43.910895   
5   SELFIES         2  512 256 0.001  37.500667  5238.600000   56.928025   
    SMILES          2  512 256 0.001  67.555903  5027.000000  123.340991   
    atom-fragSMILES 2  512 256 0.001  36.555893  4277.000000   26.514147   
    fragSMILES      2  512 256 0.001  76.008552  4441.600000  140.368444   
    t-SMILES        2  512 256 0.001  44.858667  4728.800000   42.611031   

                                          chiral               unique%  \
                                            mean         std      mean   
aug notation        hl hu  bs  lr                                        
1   SELFIES         2  512 512 0.001  763.600000   38.533103  0.934371   
    SMILES          2  512 512 0.001  818.400000   41.488553  0.918505   
    atom-fragSMILES 2  512 512 0.001  556.800000   23.931151  0.923425   
    fragSMILES      2  512 512 0.001  750.400000   47.447866  0.918221   
    t-SMILES        2  512 512 0.001  759.000000   58.096471  0.902446   
5   SELFIES         2  512 256 0.001  870.200000   59.679980  0.940885   
    SMILES          2  512 256 0.001  954.200000  132.358226  0.938249   
    atom-fragSMILES 2  512 256 0.001  545.666667   60.302018  0.917316   
    fragSMILES      2  512 256 0.001  831.200000   82.705502  0.921980   
    t-SMILES        2  512 256 0.001  814.000000   50.532168  0.908013   

                                                  novel%             chiral%  \
                                           std      mean       std      mean   
aug notation        hl hu  bs  lr                                              
1   SELFIES         2  512 512 0.001  0.006790  0.925028  0.006396  0.148162   
    SMILES          2  512 512 0.001  0.012778  0.837842  0.008778  0.177910   
    atom-fragSMILES 2  512 512 0.001  0.006709  0.841937  0.009075  0.119771   
    fragSMILES      2  512 512 0.001  0.007471  0.808590  0.006968  0.168814   
    t-SMILES        2  512 512 0.001  0.006741  0.887533  0.003963  0.160363   
5   SELFIES         2  512 256 0.001  0.006485  0.931353  0.004215  0.166076   
    SMILES          2  512 256 0.001  0.011307  0.896516  0.011472  0.189574   
    atom-fragSMILES 2  512 256 0.001  0.006792  0.779386  0.000379  0.127539   
    fragSMILES      2  512 256 0.001  0.012661  0.805715  0.014968  0.187020   
    t-SMILES        2  512 256 0.001  0.007766  0.879033  0.005323  0.172146   

                                               inducedNewFrags

In [41]:
inducedStats.to_csv('results/08_chemblScaffMetrics.csv')

In [42]:
params=[2,512,512,0.001]
params5=[2,512,256,0.001]

In [47]:
genPvals = pd.DataFrame.from_dict(
                {notation+'x1': 
                {metric:stats.ttest_ind(
                inducedGrp.get_group((1,notation,*params))[metric],
                inducedGrp.get_group((1,'fragSMILES',*params))[metric],
                ).pvalue for metric in inducedStats.columns.get_level_values(0)} for notation in ['SMILES','SELFIES','t-SMILES']
                } |
                {notation+'x5': 
                {metric:stats.ttest_ind(
                inducedGrp.get_group((5,notation,*params5))[metric],
                inducedGrp.get_group((5,'fragSMILES',*params5))[metric],
                ).pvalue for metric in inducedStats.columns.get_level_values(0)} for notation in ['SMILES','SELFIES','t-SMILES']
                }
                , orient='index'
            ).fillna(0.00001)

In [48]:
genPvals

,total,unique,novel,chiral,unique%,novel%,chiral%,inducedNewFrags,inducedNewFrags%
SMILESx1,1.222865e-01,0.883691,2.905847e-02,0.042347,0.966772,3.886867e-04,0.125945,0.00001,0.00001
SELFIESx1,1.573607e-03,0.034050,2.212652e-07,0.642110,0.007221,3.270683e-09,0.005719,0.00001,0.00001
t-SMILESx1,1.228965e-07,0.000336,1.259648e-05,0.797095,0.005058,2.035825e-09,0.224895,0.00001,0.00001
SMILESx5,5.441909e-01,0.070134,1.120501e-04,0.116053,0.064472,4.878120e-06,0.846064,0.00001,0.00001
SELFIESx5,1.000000e+00,0.017568,2.491563e-06,0.417392,0.017827,9.042331e-08,0.042876,0.00001,0.00001
t-SMILESx5,7.886681e-06,0.010166,2.355643e-03,0.701875,0.068643,6.705339e-06,0.126904,0.00001,0.00001


In [49]:
genPvals.map(lambda x: x<0.05).to_csv('results/08_chemblScaffMetrics_significant05.csv')

## saving excel files (SI)

In [57]:
df_metrics_sign = genPvals.map(lambda x: x<0.05)

In [52]:
df_metrics_1 = inducedStats.query('aug==1')
df_metrics_1.index = df_metrics_1.index.droplevel(0)
df_metrics_5 = inducedStats.query('aug==5')
df_metrics_5.index = df_metrics_5.index.droplevel(0)

In [54]:
sign_notations = ['SMILES','SELFIES','t-SMILES']

In [58]:
df_metrics_sign_1 = df_metrics_sign.loc[ [notation+'x1' for notation in sign_notations] ]
df_metrics_sign_1.index = sign_notations
df_metrics_sign_5 = df_metrics_sign.loc[ [notation+'x5' for notation in sign_notations] ]
df_metrics_sign_5.index = sign_notations

In [60]:
ResultsFromMeanStdDF(df_metrics_1, df_metrics_sign_1).to_csv('SI_tables/chembl_scaff.csv',mode='a')

/home/tox/fragsmiles/src/evaluation.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  newdf.xs(notation)[name] += '*'
/home/tox/fragsmiles/src/evaluation.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
Y

In [61]:
ResultsFromMeanStdDF(df_metrics_5, df_metrics_sign_5).to_csv('SI_tables/chembl_scaff_aug5.csv',mode='a')

/home/tox/fragsmiles/src/evaluation.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  newdf.xs(notation)[name] += '*'
/home/tox/fragsmiles/src/evaluation.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
Y